# Введение в обработку естественного языка

## Урок 1. Обработка текста

Осуществим предобработку данных с Твиттера, чтобы очищенные данные в дальнейшем
использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания. Для работы объединим train_df и test_df.
Задания:
1. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим
функцию:
- для того, чтобы найти все вхождения паттерна в тексте, необходимо
использовать re.findall(pattern, input_txt)
- для замены @user на пробел, необходимо использовать re.sub()
2. Изменим регистр твитов на нижний с помощью .lower().
3. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя
apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в
тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в
качестве ключа (сокращенного слова), то заменить ключ на значение (полную
версию слова).
4. Заменим сокращения на их полные формы, используя short_word_dict. Для этого
воспользуемся функцией, используемой в предыдущем пункте.
5. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict.
Для этого воспользуемся функцией, используемой в предыдущем пункте.
6. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.
7. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.
8. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.
9. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if
len(w)>1]).
10. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый
столбец 'tweet_token'.
11. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец
'tweet_token_filtered' без стоп-слов.
12. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим
столбец 'tweet_stemmed' после применения стемминга.
13. Применим лемматизацию к токенам с помощью
nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после
применения лемматизации.
14. Сохраним результат предобработки в pickle-файл.

In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk import tokenize as tknz
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pickle

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

True

In [2]:
df_train = pd.read_csv('train_tweets.csv')
df_test = pd.read_csv('test_tweets.csv')

In [3]:
# Объединение train и test
print(df_train.shape, df_test.shape)
df = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)
print(df.shape)

(31962, 3) (17197, 2)
(49159, 3)


In [4]:
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
df_test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [6]:
df.tail()

,id,label,tweet
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...
49157,49158,NaN,"happy, at work conference: right mindset leads..."
49158,49159,NaN,"my song ""so glad"" free download! #shoegaze ..."


In [7]:
def delete_pattern(text, pattern):
  return re.sub(pattern, ' ', text)

In [8]:
# 1. Удаление @user из всех твитов с помощью паттерна "@[\w]*"
pattern_for_username = "@[\w]*"
df['tweet'] = df['tweet'].apply(lambda x: delete_pattern(x, pattern_for_username))

In [9]:
# 2. Изменение регистра твитов на нижний с помощью .lower()
df['tweet'] = df['tweet'].apply(lambda x: x.lower())

In [10]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [11]:
def replace_words(text, correct_dict):
  new_text = ''
  for word in text.split():
    if word in correct_dict.keys():
      new_word = correct_dict[word]
    else:
      new_word = word
    new_text = new_text + ' ' + new_word
  return new_text

In [12]:
# 3. Замена сокращений с апострофами (пример: ain't, can't) на пробел с использованием apostrophe_dict. 
df['tweet'] = df['tweet'].apply(lambda x: replace_words(x, apostrophe_dict))

In [13]:
short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

In [14]:
# 4. Замена сокращений на их полные формы с использованием short_word_dict.
df['tweet'] = df['tweet'].apply(lambda x: replace_words(x, short_word_dict))

In [15]:
emoticon_dict = {
':)': 'happy',
':‑)': 'happy',
':-]': 'happy',
':-3': 'happy',
':->': 'happy',
'8-)': 'happy',
':-}': 'happy',
':o)': 'happy',
':c)': 'happy',
':^)': 'happy',
'=]': 'happy',
'=)': 'happy',
'<3': 'happy',
':-(': 'sad',
':(': 'sad',
':c': 'sad',
':<': 'sad',
':[': 'sad',
'>:[': 'sad',
':{': 'sad',
'>:(': 'sad',
':-c': 'sad',
':-< ': 'sad',
':-[': 'sad',
':-||': 'sad',
'\xF0\x9F\x98\x81': 'grinning face with smiling eyes',
'\xF0\x9F\x98\x81': 'face with tears of joy',
'\xF0\x9F\x98\x83': 'smiling face with open mouth',
'\xF0\x9F\x98\x84':	'smiling face with open mouth and smiling eyes',
'\xF0\x9F\x98\x85':	'smiling face with open mouth and cold sweat',
'\xF0\x9F\x98\x86': 'smiling face with open mouth and tightly-closed eyes',
'\xF0\x9F\x98\x89':	'winking face',
'\xF0\x9F\x98\x8A': 'smiling face with smiling eyes',
'\xF0\x9F\x98\x8B':	'face savouring delicious food',
'\xF0\x9F\x98\x8C':	'relieved face',
'\xF0\x9F\x98\x8D':	'smiling face with heart-shaped eyes',
'\xF0\x9F\x98\x8F':	'smirking face',
'\xF0\x9F\x98\x92':	'unamused face',
'\xF0\x9F\x98\x93':	'face with cold sweat',
'\xF0\x9F\x98\x94':	'pensive face',
'\xF0\x9F\x98\x96':	'confounded face',
'\xF0\x9F\x98\x98':	'face throwing a kiss',
'\xF0\x9F\x98\x9A':	'kissing face with closed eyes',
'\xF0\x9F\x98\x9C':	'face with stuck-out tongue and winking eye',
'\xF0\x9F\x98\x9D':	'face with stuck-out tongue and tightly-closed eyes',
'\xF0\x9F\x98\x9E':	'disappointed face',
'\xF0\x9F\x98\xA0':	'angry face',
'\xF0\x9F\x98\xA1':	'pouting face',
'\xF0\x9F\x98\xA2':	'crying face',
'\xF0\x9F\x98\xA3':	'persevering face',
'\xF0\x9F\x98\xA4':	'face with look of triumph',
'\xF0\x9F\x98\xA5':	'disappointed but relieved face',
'\xF0\x9F\x98\xA8':	'fearful face',
'\xF0\x9F\x98\xA9':	'weary face',
'\xF0\x9F\x98\xAA':	'sleepy face',
'\xF0\x9F\x98\xAB':	'tired face',
'\xF0\x9F\x98\xAD':	'loudly crying face',
'\xF0\x9F\x98\xB0':	'face with open mouth and cold sweat',
'\xF0\x9F\x98\xB1':	'face screaming in fear',
'\xF0\x9F\x98\xB2':	'astonished face',
'\xF0\x9F\x98\xB3':	'flushed face',
'\xF0\x9F\x98\xB5':	'dizzy face',
'\xF0\x9F\x98\xB7':	'face with medical mask',
'\xF0\x9F\x98\xB8':	'grinning cat face with smiling eyes',
'\xF0\x9F\x98\xB9':	'cat face with tears of joy',
'\xF0\x9F\x98\xBA':	'smiling cat face with open mouth',
'\xF0\x9F\x98\xBB':	'smiling cat face with heart-shaped eyes',
'\xF0\x9F\x98\xBC':	'cat face with wry smile',
'\xF0\x9F\x98\xBD':	'kissing cat face with closed eyes',
'\xF0\x9F\x98\xBE':	'pouting cat face',
'\xF0\x9F\x98\xBF':	'crying cat face',
'\xF0\x9F\x99\x80':	'weary cat face',
'\xF0\x9F\x99\x85':	'face with no good gesture',
'\xF0\x9F\x99\x86':	'face with ok gesture',
'\xF0\x9F\x99\x87':	'person bowing deeply',
'\xF0\x9F\x99\x88':	'see-no-evil monkey',
'\xF0\x9F\x99\x89':	'hear-no-evil monkey',
'\xF0\x9F\x99\x8A':	'speak-no-evil monkey',
'\xF0\x9F\x99\x8B':	'happy person raising one hand',
'\xF0\x9F\x99\x8C':	'person raising both hands in celebration',
'\xF0\x9F\x99\x8D':	'person frowning',
'\xF0\x9F\x99\x8E':	'person with pouting face',
'\xF0\x9F\x99\x8F':	'person with folded hands',
'\xE2\x9C\x82':	'black scissors',
'\xE2\x9C\x85':	'white heavy check mark',
'\xE2\x9C\x88':	'airplane',
'\xE2\x9C\x89':	'envelope',
'\xE2\x9C\x8A':	'raised fist',
'\xE2\x9C\x8B':	'raised hand',
'\xE2\x9C\x8C':	'victory hand',
'\xE2\x9C\x8F':	'pencil',
'\xE2\x9C\x92':	'black nib',
'\xE2\x9C\x94':	'heavy check mark',
'\xE2\x9C\x96':	'heavy multiplication x',
'\xE2\x9C\xA8':	'sparkles',
'\xE2\x9C\xB3':	'eight spoked asterisk',
'\xE2\x9C\xB4':	'eight pointed black star',
'\xE2\x9D\x84':	'snowflake',
'\xE2\x9D\x87':	'sparkle',
'\xE2\x9D\x8C':	'cross mark',
'\xE2\x9D\x8E':	'negative squared cross mark',
'\xE2\x9D\x93':	'black question mark ornament',
'\xE2\x9D\x94':	'white question mark ornament',
'\xE2\x9D\x95':	'white exclamation mark ornament',
'\xE2\x9D\x97':	'heavy exclamation mark symbol',
'\xE2\x9D\xA4':	'heavy black heart',
'\xE2\x9E\x95':	'heavy plus sign',
'\xE2\x9E\x96':	'heavy minus sign',
'\xE2\x9E\x97':	'heavy division sign',
'\xE2\x9E\xA1':	'black rightwards arrow',
'\xE2\x9E\xB0':	'curly loop',
'\xF0\x9F\x9A\x80':	'rocket',
'\xF0\x9F\x9A\x83':	'railway car',
'\xF0\x9F\x9A\x84':	'high-speed train',
'\xF0\x9F\x9A\x85':	'high-speed train with bullet nose',
'\xF0\x9F\x9A\x87':	'metro',
'\xF0\x9F\x9A\x89':	'station',
'\xF0\x9F\x9A\x8C':	'bus',
'\xF0\x9F\x9A\x8F':	'bus stop',
'\xF0\x9F\x9A\x91':	'ambulance',
'\xF0\x9F\x9A\x92':	'fire engine',
'\xF0\x9F\x9A\x93':	'police car',
'\xF0\x9F\x9A\x95':	'taxi',
'\xF0\x9F\x9A\x97':	'automobile',
'\xF0\x9F\x9A\x99':	'recreational vehicle',
'\xF0\x9F\x9A\x9A':	'delivery truck',
'\xF0\x9F\x9A\xA2':	'ship',
'\xF0\x9F\x9A\xA4':	'speedboat',
'\xF0\x9F\x9A\xA5':	'horizontal traffic light',
'\xF0\x9F\x9A\xA7':	'construction sign',
'\xF0\x9F\x9A\xA8':	'police cars revolving light',
'\xF0\x9F\x9A\xA9':	'triangular flag on post',
'\xF0\x9F\x9A\xAA':	'door',
'\xF0\x9F\x9A\xAB':	'no entry sign',
'\xF0\x9F\x9A\xAC':	'smoking symbol',
'\xF0\x9F\x9A\xAD':	'no smoking symbol',
'\xF0\x9F\x9A\xB2':	'bicycle',
'\xF0\x9F\x9A\xB6':	'pedestrian',
'\xF0\x9F\x9A\xB9':	'mens symbol',
'\xF0\x9F\x9A\xBA':	'womens symbol',
'\xF0\x9F\x9A\xBB':	'restroom',
'\xF0\x9F\x9A\xBC':	'baby symbol',
'\xF0\x9F\x9A\xBD':	'toilet',
'\xF0\x9F\x9A\xBE':	'water closet',
'\xF0\x9F\x9B\x80':	'bath',
'\xE2\x93\x82':	'circled latin capital letter m',
'\xF0\x9F\x85\xB0':	'negative squared latin capital letter a',
'\xF0\x9F\x85\xB1':	'negative squared latin capital letter b',
'\xF0\x9F\x85\xBE':	'negative squared latin capital letter o',
'\xF0\x9F\x85\xBF': 'negative squared latin capital letter p',
'\xF0\x9F\x86\x8E':	'negative squared ab',
'\xF0\x9F\x86\x91':	'squared cl',
'\xF0\x9F\x86\x92':	'squared cool',
'\xF0\x9F\x86\x93':	'squared free',
'\xF0\x9F\x86\x94':	'squared id',
'\xF0\x9F\x86\x95':	'squared new',
'\xF0\x9F\x86\x96':	'squared ng',
'\xF0\x9F\x86\x97':	'squared ok',
'\xF0\x9F\x86\x98':	'squared sos',
'\xF0\x9F\x86\x99':	'squared up with exclamation mark',
'\xF0\x9F\x86\x9A':	'squared vs',
'\xF0\x9F\x87\xA9\xF0\x9F\x87\xAA':	'regional indicator symbol letter d + regional indicator symbol letter e',
'\xF0\x9F\x87\xAC\xF0\x9F\x87\xA7':	'regional indicator symbol letter g + regional indicator symbol letter b',
'\xF0\x9F\x87\xA8\xF0\x9F\x87\xB3':	'regional indicator symbol letter c + regional indicator symbol letter n',
'\xF0\x9F\x87\xAF\xF0\x9F\x87\xB5':	'regional indicator symbol letter j + regional indicator symbol letter p',
'\xF0\x9F\x87\xAB\xF0\x9F\x87\xB7':	'regional indicator symbol letter f + regional indicator symbol letter r',
'\xF0\x9F\x87\xB0\xF0\x9F\x87\xB7':	'regional indicator symbol letter k + regional indicator symbol letter r',
'\xF0\x9F\x87\xAA\xF0\x9F\x87\xB8':	'regional indicator symbol letter e + regional indicator symbol letter s',
'\xF0\x9F\x87\xAE\xF0\x9F\x87\xB9':	'regional indicator symbol letter i + regional indicator symbol letter t',
'\xF0\x9F\x87\xB7\xF0\x9F\x87\xBA':	'regional indicator symbol letter r + regional indicator symbol letter u',
'\xF0\x9F\x87\xBA\xF0\x9F\x87\xB8':	'regional indicator symbol letter u + regional indicator symbol letter s',
'\xF0\x9F\x88\x81':	'squared katakana koko',
'\xF0\x9F\x88\x82':	'squared katakana sa',
'\xF0\x9F\x88\x9A':	'squared cjk unified ideograph-7121',
'\xF0\x9F\x88\xAF':	'squared cjk unified ideograph-6307',
'\xF0\x9F\x88\xB2':	'squared cjk unified ideograph-7981',
'\xF0\x9F\x88\xB3':	'squared cjk unified ideograph-7a7a',
'\xF0\x9F\x88\xB4':	'squared cjk unified ideograph-5408',
'\xF0\x9F\x88\xB5':	'squared cjk unified ideograph-6e80',
'\xF0\x9F\x88\xB6':	'squared cjk unified ideograph-6709',
'\xF0\x9F\x88\xB7':	'squared cjk unified ideograph-6708',
'\xF0\x9F\x88\xB8':	'squared cjk unified ideograph-7533',
'\xF0\x9F\x88\xB9':	'squared cjk unified ideograph-5272',
'\xF0\x9F\x88\xBA':	'squared cjk unified ideograph-55b6',
'\xF0\x9F\x89\x90':	'circled ideograph advantage',
'\xF0\x9F\x89\x91':	'circled ideograph accept',
'\xC2\xA9':	'copyright sign',
'\xC2\xAE':	'registered sign',
'\xE2\x80\xBC':	'double exclamation mark',
'\xE2\x81\x89':	'exclamation question mark',
'\x23\xE2\x83\xA3':	'number sign + combining enclosing keycap',
'\x38\xE2\x83\xA3':	'digit eight + combining enclosing keycap',
'\x39\xE2\x83\xA3':	'digit nine + combining enclosing keycap',
'\x37\xE2\x83\xA3':	'digit seven + combining enclosing keycap',
'\x30\xE2\x83\xA3':	'digit zero + combining enclosing keycap',
'\x36\xE2\x83\xA3':	'digit six + combining enclosing keycap',
'\x35\xE2\x83\xA3':	'digit five + combining enclosing keycap',
'\x34\xE2\x83\xA3':	'digit four + combining enclosing keycap', 
'\x33\xE2\x83\xA3':	'digit three + combining enclosing keycap',
'\x32\xE2\x83\xA3':	'digit two + combining enclosing keycap',
'\x31\xE2\x83\xA3':	'digit one + combining enclosing keycap',
'\xE2\x84\xA2':	'trade mark sign',
'\xE2\x84\xB9':	'information source',
'\xE2\x86\x94':	'left right arrow',
'\xE2\x86\x95':	'up down arrow',
'\xE2\x86\x96':	'north west arrow',
'\xE2\x86\x97':	'north east arrow',
'\xE2\x86\x98':	'south east arrow',
'\xE2\x86\x99':	'south west arrow',
'\xE2\x86\xA9':	'leftwards arrow with hook',
'\xE2\x86\xAA':	'rightwards arrow with hook',
'\xE2\x8C\x9A':	'watch',
'\xE2\x8C\x9B':	'hourglass',
'\xE2\x8F\xA9':	'black right-pointing double triangle',
'\xE2\x8F\xAA':	'black left-pointing double triangle',
'\xE2\x8F\xAB':	'black up-pointing double triangle',
'\xE2\x8F\xAC':	'black down-pointing double triangle',
'\xE2\x8F\xB0':	'alarm clock',
'\xE2\x8F\xB3':	'hourglass with flowing sand',
'\xE2\x96\xAA':	'black small square',
'\xE2\x96\xAB':	'white small square',
'\xE2\x96\xB6':	'black right-pointing triangle',
'\xE2\x97\x80':	'black left-pointing triangle',
'\xE2\x97\xBB':	'white medium square',
'\xE2\x97\xBC':	'black medium square',
'\xE2\x97\xBD':	'white medium small square',
'\xE2\x97\xBE':	'black medium small square',
'\xE2\x98\x80':	'black sun with rays',
'\xE2\x98\x81':	'cloud',
'\xE2\x98\x8E':	'black telephone',
'\xE2\x98\x91':	'ballot box with check',
'\xE2\x98\x94':	'umbrella with rain drops',
'\xE2\x98\x95':	'hot beverage',
'\xE2\x98\x9D':	'white up pointing index',
'\xE2\x98\xBA':	'white smiling face',
'\xE2\x99\x88':	'aries',
'\xE2\x99\x89':	'taurus',
'\xE2\x99\x8A':	'gemini',
'\xE2\x99\x8B':	'cancer',
'\xE2\x99\x8C':	'leo',
'\xE2\x99\x8D':	'virgo',
'\xE2\x99\x8E':	'libra',
'\xE2\x99\x8F':	'scorpius',
'\xE2\x99\x90':	'sagittarius',
'\xE2\x99\x91':	'capricorn',
'\xE2\x99\x92':	'aquarius',
'\xE2\x99\x93':	'pisces',
'\xE2\x99\xA0':	'black spade suit',
'\xE2\x99\xA3':	'black club suit',
'\xE2\x99\xA5':	'black heart suit', 
'\xE2\x99\xA6':	'black diamond suit',
'\xE2\x99\xA8':	'hot springs',
'\xE2\x99\xBB':	'black universal recycling symbol',
'\xE2\x99\xBF':	'wheelchair symbol',
'\xE2\x9A\x93':	'anchor',
'\xE2\x9A\xA0':	'warning sign',
'\xE2\x9A\xA1':	'high voltage sign',
'\xE2\x9A\xAA':	'medium white circle',
'\xE2\x9A\xAB':	'medium black circle',
'\xE2\x9A\xBD':	'soccer ball',
'\xE2\x9A\xBE':	'baseball',
'\xE2\x9B\x84':	'snowman without snow',
'\xE2\x9B\x85':	'sun behind cloud',
'\xE2\x9B\x8E':	'ophiuchus',
'\xE2\x9B\x94':	'no entry',
'\xE2\x9B\xAA':	'church',
'\xE2\x9B\xB2':	'fountain',
'\xE2\x9B\xB3':	'flag in hole',
'\xE2\x9B\xB5':	'sailboat',
'\xE2\x9B\xBA':	'tent',
'\xE2\x9B\xBD':	'fuel pump',
'\xE2\xA4\xB4':	'arrow pointing rightwards then curving upwards',
'\xE2\xA4\xB5':	'arrow pointing rightwards then curving downwards',
'\xE2\xAC\x85':	'leftwards black arrow',
'\xE2\xAC\x86':	'upwards black arrow',
'\xE2\xAC\x87':	'downwards black arrow',
'\xE2\xAC\x9B':	'black large square',
'\xE2\xAC\x9C':	'white large square',
'\xE2\xAD\x90':	'white medium star',
'\xE2\xAD\x95':	'heavy large circle',
'\xE3\x80\xB0':	'wavy dash',
'\xE3\x80\xBD':	'part alternation mark',
'\xE3\x8A\x97':	'circled ideograph congratulation',
'\xE3\x8A\x99':	'circled ideograph secret',
'\xF0\x9F\x80\x84':	'mahjong tile red dragon',
'\xF0\x9F\x83\x8F':	'playing card black joker',
'\xF0\x9F\x8C\x80':	'cyclone',
'\xF0\x9F\x8C\x81':	'foggy',
'\xF0\x9F\x8C\x82':	'closed umbrella',
'\xF0\x9F\x8C\x83':	'night with stars',
'\xF0\x9F\x8C\x84':	'sunrise over mountains',
'\xF0\x9F\x8C\x85':	'sunrise',
'\xF0\x9F\x8C\x86':	'cityscape at dusk',
'\xF0\x9F\x8C\x87':	'sunset over buildings',
'\xF0\x9F\x8C\x88':	'rainbow',
'\xF0\x9F\x8C\x89':	'bridge at night',
'\xF0\x9F\x8C\x8A':	'water wave',
'\xF0\x9F\x8C\x8B':	'volcano',
'\xF0\x9F\x8C\x8C':	'milky way',
'\xF0\x9F\x8C\x8F':	'earth globe asia-australia',
'\xF0\x9F\x8C\x91':	'new moon symbol',
'\xF0\x9F\x8C\x93':	'first quarter moon symbol',
'\xF0\x9F\x8C\x94':	'waxing gibbous moon symbol',
'\xF0\x9F\x8C\x95':	'full moon symbol',
'\xF0\x9F\x8C\x99':	'crescent moon',
'\xF0\x9F\x8C\x9B':	'first quarter moon with face',
'\xF0\x9F\x8C\x9F':	'glowing star',
'\xF0\x9F\x8C\xA0':	'shooting star',
'\xF0\x9F\x8C\xB0':	'chestnut',
'\xF0\x9F\x8C\xB1':	'seedling',
'\xF0\x9F\x8C\xB4':	'palm tree',
'\xF0\x9F\x8C\xB5':	'cactus',
'\xF0\x9F\x8C\xB7':	'tulip',
'\xF0\x9F\x8C\xB8':	'cherry blossom',
'\xF0\x9F\x8C\xB9':	'rose',
'\xF0\x9F\x8C\xBA':	'hibiscus',
'\xF0\x9F\x8C\xBB':	'sunflower',
'\xF0\x9F\x8C\xBC':	'blossom',
'\xF0\x9F\x8C\xBD':	'ear of maize',
'\xF0\x9F\x8C\xBE':	'ear of rice',
'\xF0\x9F\x8C\xBF':	'herb',
'\xF0\x9F\x8D\x80':	'four leaf clover',
'\xF0\x9F\x8D\x81':	'maple leaf',
'\xF0\x9F\x8D\x82':	'fallen leaf',
'\xF0\x9F\x8D\x83':	'leaf fluttering in wind',
'\xF0\x9F\x8D\x84':	'mushroom',
'\xF0\x9F\x8D\x85':	'tomato',
'\xF0\x9F\x8D\x86':	'aubergine',
'\xF0\x9F\x8D\x87':	'grapes',
'\xF0\x9F\x8D\x88':	'melon',
'\xF0\x9F\x8D\x89':	'watermelon',
'\xF0\x9F\x8D\x8A':	'tangerine',
'\xF0\x9F\x8D\x8C':	'banana',
'\xF0\x9F\x8D\x8D':	'pineapple',
'\xF0\x9F\x8D\x8E':	'red apple',
'\xF0\x9F\x8D\x8F':	'green apple',
'\xF0\x9F\x8D\x91':	'peach',
'\xF0\x9F\x8D\x92':	'cherries',
'\xF0\x9F\x8D\x93':	'strawberry',
'\xF0\x9F\x8D\x94':	'hamburger',
'\xF0\x9F\x8D\x95':	'slice of pizza',
'\xF0\x9F\x8D\x96':	'meat on bone',
'\xF0\x9F\x8D\x97':	'poultry leg',
'\xF0\x9F\x8D\x98':	'rice cracker',
'\xF0\x9F\x8D\x99':	'rice ball',
'\xF0\x9F\x8D\x9A':	'cooked rice',
'\xF0\x9F\x8D\x9B':	'curry and rice',
'\xF0\x9F\x8D\x9C':	'steaming bowl',
'\xF0\x9F\x8D\x9D':	'spaghetti',
'\xF0\x9F\x8D\x9E':	'bread',
'\xF0\x9F\x8D\x9F':	'french fries',
'\xF0\x9F\x8D\xA0':	'roasted sweet potato',
'\xF0\x9F\x8D\xA1':	'dango',
'\xF0\x9F\x8D\xA2':	'oden',
'\xF0\x9F\x8D\xA3':	'sushi',
'\xF0\x9F\x8D\xA4':	'fried shrimp',
'\xF0\x9F\x8D\xA5':	'fish cake with swirl design',
'\xF0\x9F\x8D\xA6':	'soft ice cream',
'\xF0\x9F\x8D\xA7':	'shaved ice',
'\xF0\x9F\x8D\xA8':	'ice cream',
'\xF0\x9F\x8D\xA9':	'doughnut',
'\xF0\x9F\x8D\xAA':	'cookie',
'\xF0\x9F\x8D\xAB':	'chocolate bar',
'\xF0\x9F\x8D\xAC':	'candy',
'\xF0\x9F\x8D\xAD':	'lollipop',
'\xF0\x9F\x8D\xAE':	'custard',
'\xF0\x9F\x8D\xAF':	'honey pot',
'\xF0\x9F\x8D\xB0':	'shortcake',
'\xF0\x9F\x8D\xB1':	'bento box',
'\xF0\x9F\x8D\xB2':	'pot of food',
'\xF0\x9F\x8D\xB3':	'cooking',
'\xF0\x9F\x8D\xB4':	'fork and knife',
'\xF0\x9F\x8D\xB5':	'teacup without handle',
'\xF0\x9F\x8D\xB6':	'sake bottle and cup',
'\xF0\x9F\x8D\xB7':	'wine glass',
'\xF0\x9F\x8D\xB8':	'cocktail glass',
'\xF0\x9F\x8D\xB9':	'tropical drink',
'\xF0\x9F\x8D\xBA':	'beer mug',
'\xF0\x9F\x8D\xBB':	'clinking beer mugs',
'\xF0\x9F\x8E\x80':	'ribbon',
'\xF0\x9F\x8E\x81':	'wrapped present',
'\xF0\x9F\x8E\x82':	'birthday cake',
'\xF0\x9F\x8E\x83':	'jack-o-lantern',
'\xF0\x9F\x8E\x84':	'christmas tree',
'\xF0\x9F\x8E\x85':	'father christmas',
'\xF0\x9F\x8E\x86':	'fireworks',
'\xF0\x9F\x8E\x87':	'firework sparkler',
'\xF0\x9F\x8E\x88':	'balloon',
'\xF0\x9F\x8E\x89':	'party popper',
'\xF0\x9F\x8E\x8A':	'confetti ball',
'\xF0\x9F\x8E\x8B':	'tanabata tree',
'\xF0\x9F\x8E\x8C':	'crossed flags',
'\xF0\x9F\x8E\x8D':	'pine decoration',
'\xF0\x9F\x8E\x8E':	'japanese dolls',
'\xF0\x9F\x8E\x8F':	'carp streamer',
'\xF0\x9F\x8E\x90':	'wind chime',
'\xF0\x9F\x8E\x91':	'moon viewing ceremony',
'\xF0\x9F\x8E\x92':	'school satchel',
'\xF0\x9F\x8E\x93':	'graduation cap',
'\xF0\x9F\x8E\xA0':	'carousel horse',
'\xF0\x9F\x8E\xA1':	'ferris wheel',
'\xF0\x9F\x8E\xA2':	'roller coaster',
'\xF0\x9F\x8E\xA3':	'fishing pole and fish',
'\xF0\x9F\x8E\xA4':	'microphone',
'\xF0\x9F\x8E\xA5':	'movie camera',
'\xF0\x9F\x8E\xA6':	'cinema', 
'\xF0\x9F\x8E\xA7':	'headphone',
'\xF0\x9F\x8E\xA8':	'artist palette',
'\xF0\x9F\x8E\xA9':	'top hat',
'\xF0\x9F\x8E\xAA':	'circus tent',
'\xF0\x9F\x8E\xAB':	'ticket',
'\xF0\x9F\x8E\xAC':	'clapper board',
'\xF0\x9F\x8E\xAD':	'performing arts',
'\xF0\x9F\x8E\xAE':	'video game',
'\xF0\x9F\x8E\xAF':	'direct hit',
'\xF0\x9F\x8E\xB0':	'slot machine',
'\xF0\x9F\x8E\xB1':	'billiards',
'\xF0\x9F\x8E\xB2':	'game die',
'\xF0\x9F\x8E\xB3':	'bowling',
'\xF0\x9F\x8E\xB4':	'flower playing cards',
'\xF0\x9F\x8E\xB5':	'musical note',
'\xF0\x9F\x8E\xB6':	'multiple musical notes',
'\xF0\x9F\x8E\xB7':	'saxophone',
'\xF0\x9F\x8E\xB8':	'guitar',
'\xF0\x9F\x8E\xB9':	'musical keyboard',
'\xF0\x9F\x8E\xBA':	'trumpet',
'\xF0\x9F\x8E\xBB':	'violin',
'\xF0\x9F\x8E\xBC':	'musical score',
'\xF0\x9F\x8E\xBD':	'running shirt with sash',
'\xF0\x9F\x8E\xBE':	'tennis racquet and ball',
'\xF0\x9F\x8E\xBF':	'ski and ski boot',
'\xF0\x9F\x8F\x80':	'basketball and hoop',
'\xF0\x9F\x8F\x81':	'chequered flag',
'\xF0\x9F\x8F\x82':	'snowboarder',
'\xF0\x9F\x8F\x83':	'runner',
'\xF0\x9F\x8F\x84':	'surfer',
'\xF0\x9F\x8F\x86':	'trophy',
'\xF0\x9F\x8F\x88':	'american football',
'\xF0\x9F\x8F\x8A':	'swimmer',
'\xF0\x9F\x8F\xA0':	'house building',
'\xF0\x9F\x8F\xA1':	'house with garden',
'\xF0\x9F\x8F\xA2':	'office building',
'\xF0\x9F\x8F\xA3':	'japanese post office',
'\xF0\x9F\x8F\xA5':	'hospital',
'\xF0\x9F\x8F\xA6':	'bank',
'\xF0\x9F\x8F\xA7':	'automated teller machine',
'\xF0\x9F\x8F\xA8':	'hotel',
'\xF0\x9F\x8F\xA9':	'love hotel',
'\xF0\x9F\x8F\xAA':	'convenience store',
'\xF0\x9F\x8F\xAB':	'school',
'\xF0\x9F\x8F\xAC':	'department store',
'\xF0\x9F\x8F\xAD':	'factory',
'\xF0\x9F\x8F\xAE':	'izakaya lantern',
'\xF0\x9F\x8F\xAF':	'japanese castle',
'\xF0\x9F\x8F\xB0':	'european castle',
'\xF0\x9F\x90\x8C':	'snail',
'\xF0\x9F\x90\x8D':	'snake',
'\xF0\x9F\x90\x8E':	'horse',
'\xF0\x9F\x90\x91':	'sheep',
'\xF0\x9F\x90\x92':	'monkey',
'\xF0\x9F\x90\x94':	'chicken',
'\xF0\x9F\x90\x97':	'boar',
'\xF0\x9F\x90\x98':	'elephant',
'\xF0\x9F\x90\x99':	'octopus',
'\xF0\x9F\x90\x9A':	'spiral shell',
'\xF0\x9F\x90\x9B':	'bug',
'\xF0\x9F\x90\x9C':	'ant',
'\xF0\x9F\x90\x9D':	'honeybee',
'\xF0\x9F\x90\x9E':	'lady beetle',
'\xF0\x9F\x90\x9F':	'fish',
'\xF0\x9F\x90\xA0':	'tropical fish',
'\xF0\x9F\x90\xA1':	'blowfish',
'\xF0\x9F\x90\xA2':	'turtle',
'\xF0\x9F\x90\xA3':	'hatching chick',
'\xF0\x9F\x90\xA4':	'baby chick',
'\xF0\x9F\x90\xA5':	'front-facing baby chick',
'\xF0\x9F\x90\xA6':	'bird',
'\xF0\x9F\x90\xA7':	'penguin',
'\xF0\x9F\x90\xA8':	'koala',
'\xF0\x9F\x90\xA9':	'poodle',
'\xF0\x9F\x90\xAB':	'bactrian camel',
'\xF0\x9F\x90\xAC':	'dolphin',
'\xF0\x9F\x90\xAD':	'mouse face',
'\xF0\x9F\x90\xAE':	'cow face',
'\xF0\x9F\x90\xAF':	'tiger face',
'\xF0\x9F\x90\xB0':	'rabbit face',
'\xF0\x9F\x90\xB1':	'cat face',
'\xF0\x9F\x90\xB2':	'dragon face',
'\xF0\x9F\x90\xB3':	'spouting whale',
'\xF0\x9F\x90\xB4':	'horse face',
'\xF0\x9F\x90\xB5':	'monkey face',
'\xF0\x9F\x90\xB6':	'dog face',
'\xF0\x9F\x90\xB7':	'pig face',
'\xF0\x9F\x90\xB8':	'frog face',
'\xF0\x9F\x90\xB9':	'hamster face',
'\xF0\x9F\x90\xBA':	'wolf face',
'\xF0\x9F\x90\xBB':	'bear face',
'\xF0\x9F\x90\xBC':	'panda face',
'\xF0\x9F\x90\xBD':	'pig nose',
'\xF0\x9F\x90\xBE':	'paw prints',
'\xF0\x9F\x91\x80':	'eyes',
'\xF0\x9F\x91\x82':	'ear',
'\xF0\x9F\x91\x83':	'nose',
'\xF0\x9F\x91\x84':	'mouth',
'\xF0\x9F\x91\x85':	'tongue',
'\xF0\x9F\x91\x86':	'white up pointing backhand index',
'\xF0\x9F\x91\x87':	'white down pointing backhand index',
'\xF0\x9F\x91\x88':	'white left pointing backhand index',
'\xF0\x9F\x91\x89':	'white right pointing backhand index',
'\xF0\x9F\x91\x8A':	'fisted hand sign',
'\xF0\x9F\x91\x8B':	'waving hand sign',
'\xF0\x9F\x91\x8C':	'ok hand sign',
'\xF0\x9F\x91\x8D':	'thumbs up sign',
'\xF0\x9F\x91\x8E':	'thumbs down sign',
'\xF0\x9F\x91\x8F':	'clapping hands sign',
'\xF0\x9F\x91\x90':	'open hands sign',
'\xF0\x9F\x91\x91':	'crown',
'\xF0\x9F\x91\x92':	'womans hat',
'\xF0\x9F\x91\x93':	'eyeglasses',
'\xF0\x9F\x91\x94':	'necktie',
'\xF0\x9F\x91\x95': 't-shirt',
'\xF0\x9F\x91\x96':	'jeans',
'\xF0\x9F\x91\x97':	'dress',
'\xF0\x9F\x91\x98':	'kimono',
'\xF0\x9F\x91\x99':	'bikini',
'\xF0\x9F\x91\x9A':	'womans clothes',
'\xF0\x9F\x91\x9B':	'purse',
'\xF0\x9F\x91\x9C':	'handbag',
'\xF0\x9F\x91\x9D':	'pouch',
'\xF0\x9F\x91\x9E':	'mans shoe',
'\xF0\x9F\x91\x9F':	'athletic shoe',
'\xF0\x9F\x91\xA0':	'high-heeled shoe',
'\xF0\x9F\x91\xA1':	'womans sandal',
'\xF0\x9F\x91\xA2':	'womans boots',
'\xF0\x9F\x91\xA3':	'footprints',
'\xF0\x9F\x91\xA4':	'bust in silhouette',
'\xF0\x9F\x91\xA6':	'boy',
'\xF0\x9F\x91\xA7':	'girl',
'\xF0\x9F\x91\xA8':	'man',
'\xF0\x9F\x91\xA9':	'woman',
'\xF0\x9F\x91\xAA':	'family',
'\xF0\x9F\x91\xAB':	'man and woman holding hands',
'\xF0\x9F\x91\xAE':	'police officer',
'\xF0\x9F\x91\xAF':	'woman with bunny ears',
'\xF0\x9F\x91\xB0':	'bride with veil',
'\xF0\x9F\x91\xB1':	'person with blond hair',
'\xF0\x9F\x91\xB2':	'man with gua pi mao',
'\xF0\x9F\x91\xB3':	'man with turban',
'\xF0\x9F\x91\xB4':	'older man',
'\xF0\x9F\x91\xB5':	'older woman',
'\xF0\x9F\x91\xB6':	'baby',
'\xF0\x9F\x91\xB7':	'construction worker',
'\xF0\x9F\x91\xB8':	'princess',
'\xF0\x9F\x91\xB9':	'japanese ogre',
'\xF0\x9F\x91\xBA':	'japanese goblin',
'\xF0\x9F\x91\xBB':	'ghost',
'\xF0\x9F\x91\xBC':	'baby angel', 
'\xF0\x9F\x91\xBD':	'extraterrestrial alien',
'\xF0\x9F\x91\xBE':	'alien monster',
'\xF0\x9F\x91\xBF':	'imp',
'\xF0\x9F\x92\x80':	'skull',
'\xF0\x9F\x92\x81':	'information desk person',
'\xF0\x9F\x92\x82':	'guardsman',
'\xF0\x9F\x92\x83':	'dancer',
'\xF0\x9F\x92\x84':	'lipstick',
'\xF0\x9F\x92\x85':	'nail polish',
'\xF0\x9F\x92\x86':	'face massage',
'\xF0\x9F\x92\x87':	'haircut',
'\xF0\x9F\x92\x88':	'barber pole',
'\xF0\x9F\x92\x89':	'syringe',
'\xF0\x9F\x92\x8A':	'pill',
'\xF0\x9F\x92\x8B':	'kiss mark',
'\xF0\x9F\x92\x8C':	'love letter',
'\xF0\x9F\x92\x8D':	'ring',
'\xF0\x9F\x92\x8E':	'gem stone',
'\xF0\x9F\x92\x8F':	'kiss',
'\xF0\x9F\x92\x90':	'bouquet',
'\xF0\x9F\x92\x91':	'couple with heart',
'\xF0\x9F\x92\x92':	'wedding',
'\xF0\x9F\x92\x93':	'beating heart',
'\xF0\x9F\x92\x94':	'broken heart',
'\xF0\x9F\x92\x95':	'two hearts', 
'\xF0\x9F\x92\x96': 'sparkling heart',
'\xF0\x9F\x92\x97':	'growing heart',
'\xF0\x9F\x92\x98':	'heart with arrow',
'\xF0\x9F\x92\x99':	'blue heart',
'\xF0\x9F\x92\x9A':	'green heart',
'\xF0\x9F\x92\x9B':	'yellow heart',
'\xF0\x9F\x92\x9C':	'purple heart',
'\xF0\x9F\x92\x9D':	'heart with ribbon',
'\xF0\x9F\x92\x9E':	'revolving hearts',
'\xF0\x9F\x92\x9F':	'heart decoration',
'\xF0\x9F\x92\xA0':	'diamond shape with a dot inside',
'\xF0\x9F\x92\xA1':	'electric light bulb',
'\xF0\x9F\x92\xA2':	'anger symbol',
'\xF0\x9F\x92\xA3':	'bomb',
'\xF0\x9F\x92\xA4':	'sleeping symbol',
'\xF0\x9F\x92\xA5':	'collision symbol',
'\xF0\x9F\x92\xA6':	'splashing sweat symbol',
'\xF0\x9F\x92\xA7':	'droplet',
'\xF0\x9F\x92\xA8':	'dash symbol',
'\xF0\x9F\x92\xA9':	'pile of poo',
'\xF0\x9F\x92\xAA':	'flexed biceps',
'\xF0\x9F\x92\xAB':	'dizzy symbol',
'\xF0\x9F\x92\xAC':	'speech balloon',
'\xF0\x9F\x92\xAE':	'white flower',
'\xF0\x9F\x92\xAF':	'hundred points symbol',
'\xF0\x9F\x92\xB0':	'money bag',
'\xF0\x9F\x92\xB1':	'currency exchange',
'\xF0\x9F\x92\xB2':	'heavy dollar sign',
'\xF0\x9F\x92\xB3':	'credit card',
'\xF0\x9F\x92\xB4':	'banknote with yen sign',
'\xF0\x9F\x92\xB5':	'banknote with dollar sign',
'\xF0\x9F\x92\xB8':	'money with wings',
'\xF0\x9F\x92\xB9':	'chart with upwards trend and yen sign',
'\xF0\x9F\x92\xBA':	'seat',
'\xF0\x9F\x92\xBB':	'personal computer',
'\xF0\x9F\x92\xBC':	'briefcase',
'\xF0\x9F\x92\xBD':	'minidisc',
'\xF0\x9F\x92\xBE':	'floppy disk',
'\xF0\x9F\x92\xBF':	'optical disc',
'\xF0\x9F\x93\x80':	'dvd',
'\xF0\x9F\x93\x81':	'file folder',
'\xF0\x9F\x93\x82':	'open file folder',
'\xF0\x9F\x93\x83':	'page with curl',
'\xF0\x9F\x93\x84':	'page facing up',
'\xF0\x9F\x93\x85':	'calendar',
'\xF0\x9F\x93\x86':	'tear-off calendar',
'\xF0\x9F\x93\x87':	'card index',
'\xF0\x9F\x93\x88':	'chart with upwards trend',
'\xF0\x9F\x93\x89':	'chart with downwards trend',
'\xF0\x9F\x93\x8A':	'bar chart',
'\xF0\x9F\x93\x8B':	'clipboard',
'\xF0\x9F\x93\x8C':	'pushpin',
'\xF0\x9F\x93\x8D':	'round pushpin',
'\xF0\x9F\x93\x8E':	'paperclip',
'\xF0\x9F\x93\x8F':	'straight ruler',
'\xF0\x9F\x93\x90':	'triangular ruler',
'\xF0\x9F\x93\x91':	'bookmark tabs',
'\xF0\x9F\x93\x92':	'ledger',
'\xF0\x9F\x93\x93':	'notebook',
'\xF0\x9F\x93\x94':	'notebook with decorative cover',
'\xF0\x9F\x93\x95':	'closed book',
'\xF0\x9F\x93\x96':	'open book',
'\xF0\x9F\x93\x97':	'green book',
'\xF0\x9F\x93\x98':	'blue book',
'\xF0\x9F\x93\x99':	'orange book',
'\xF0\x9F\x93\x9A':	'books',
'\xF0\x9F\x93\x9B':	'name badge',
'\xF0\x9F\x93\x9C':	'scroll',
'\xF0\x9F\x93\x9D':	'memo',
'\xF0\x9F\x93\x9E':	'telephone receiver',
'\xF0\x9F\x93\x9F':	'pager',
'\xF0\x9F\x93\xA0':	'fax machine',
'\xF0\x9F\x93\xA1':	'satellite antenna',
'\xF0\x9F\x93\xA2':	'public address loudspeaker',
'\xF0\x9F\x93\xA3':	'cheering megaphone',
'\xF0\x9F\x93\xA4':	'outbox tray',
'\xF0\x9F\x93\xA5':	'inbox tray', 
'\xF0\x9F\x93\xA6':	'package',
'\xF0\x9F\x93\xA7':	'e-mail symbol',
'\xF0\x9F\x93\xA8':	'incoming envelope',
'\xF0\x9F\x93\xA9':	'envelope with downwards arrow above',
'\xF0\x9F\x93\xAA':	'closed mailbox with lowered flag',
'\xF0\x9F\x93\xAB':	'closed mailbox with raised flag',
'\xF0\x9F\x93\xAE':	'postbox',
'\xF0\x9F\x93\xB0':	'newspaper',
'\xF0\x9F\x93\xB1':	'mobile phone',
'\xF0\x9F\x93\xB2':	'mobile phone with rightwards arrow at left',
'\xF0\x9F\x93\xB3':	'vibration mode',
'\xF0\x9F\x93\xB4':	'mobile phone off',
'\xF0\x9F\x93\xB6':	'antenna with bars',
'\xF0\x9F\x93\xB7':	'camera',
'\xF0\x9F\x93\xB9':	'video camera',
'\xF0\x9F\x93\xBA':	'television',
'\xF0\x9F\x93\xBB':	'radio',
'\xF0\x9F\x93\xBC':	'videocassette',
'\xF0\x9F\x94\x83':	'clockwise downwards and upwards open circle arrows',
'\xF0\x9F\x94\x8A':	'speaker with three sound waves',
'\xF0\x9F\x94\x8B':	'battery',
'\xF0\x9F\x94\x8C':	'electric plug',
'\xF0\x9F\x94\x8D':	'left-pointing magnifying glass',
'\xF0\x9F\x94\x8E':	'right-pointing magnifying glass',
'\xF0\x9F\x94\x8F':	'lock with ink pen',
'\xF0\x9F\x94\x90':	'closed lock with key',
'\xF0\x9F\x94\x91':	'key',
'\xF0\x9F\x94\x92':	'lock',
'\xF0\x9F\x94\x93':	'open lock',
'\xF0\x9F\x94\x94':	'bell',
'\xF0\x9F\x94\x96':	'bookmark',
'\xF0\x9F\x94\x97':	'link symbol',
'\xF0\x9F\x94\x98':	'radio button',
'\xF0\x9F\x94\x99':	'back with leftwards arrow above',
'\xF0\x9F\x94\x9A':	'end with leftwards arrow above',
'\xF0\x9F\x94\x9B':	'on with exclamation mark with left right arrow above',
'\xF0\x9F\x94\x9C':	'soon with rightwards arrow above',
'\xF0\x9F\x94\x9D':	'top with upwards arrow above',
'\xF0\x9F\x94\x9E':	'no one under eighteen symbol',
'\xF0\x9F\x94\x9F':	'keycap ten',
'\xF0\x9F\x94\xA0':	'input symbol for latin capital letters',
'\xF0\x9F\x94\xA1':	'input symbol for latin small letters',
'\xF0\x9F\x94\xA2':	'input symbol for numbers',
'\xF0\x9F\x94\xA3':	'input symbol for symbols',
'\xF0\x9F\x94\xA4':	'input symbol for latin letters',
'\xF0\x9F\x94\xA5':	'fire',
'\xF0\x9F\x94\xA6':	'electric torch',
'\xF0\x9F\x94\xA7':	'wrench',
'\xF0\x9F\x94\xA8':	'hammer',
'\xF0\x9F\x94\xA9':	'nut and bolt',
'\xF0\x9F\x94\xAA':	'hocho',
'\xF0\x9F\x94\xAB':	'pistol',
'\xF0\x9F\x94\xAE':	'crystal ball',
'\xF0\x9F\x94\xAF':	'six pointed star with middle dot',
'\xF0\x9F\x94\xB0':	'japanese symbol for beginner',
'\xF0\x9F\x94\xB1':	'trident emblem',
'\xF0\x9F\x94\xB2':	'black square button',
'\xF0\x9F\x94\xB3':	'white square button',
'\xF0\x9F\x94\xB4':	'large red circle',
'\xF0\x9F\x94\xB5':	'large blue circle',
'\xF0\x9F\x94\xB6':	'large orange diamond',
'\xF0\x9F\x94\xB7':	'large blue diamond',
'\xF0\x9F\x94\xB8':	'small orange diamond',
'\xF0\x9F\x94\xB9':	'small blue diamond',
'\xF0\x9F\x94\xBA':	'up-pointing red triangle',
'\xF0\x9F\x94\xBB':	'down-pointing red triangle',
'\xF0\x9F\x94\xBC':	'up-pointing small red triangle',
'\xF0\x9F\x94\xBD':	'down-pointing small red triangle',
'\xF0\x9F\x95\x90':	'clock face one oclock',
'\xF0\x9F\x95\x91':	'clock face two oclock',
'\xF0\x9F\x95\x92':	'clock face three oclock',
'\xF0\x9F\x95\x93':	'clock face four oclock',
'\xF0\x9F\x95\x94':	'clock face five oclock',
'\xF0\x9F\x95\x95':	'clock face six oclock',
'\xF0\x9F\x95\x96':	'clock face seven oclock',
'\xF0\x9F\x95\x97':	'clock face eight oclock',
'\xF0\x9F\x95\x98':	'clock face nine oclock',
'\xF0\x9F\x95\x99':	'clock face ten oclock',
'\xF0\x9F\x95\x9A':	'clock face eleven oclock',
'\xF0\x9F\x95\x9B':	'clock face twelve oclock',
'\xF0\x9F\x97\xBB':	'mount fuji',
'\xF0\x9F\x97\xBC':	'tokyo tower',
'\xF0\x9F\x97\xBD':	'statue of liberty',
'\xF0\x9F\x97\xBE':	'silhouette of japan',
'\xF0\x9F\x97\xBF':	'moyai',
'\xF0\x9F\x98\x80':	'grinning face',
'\xF0\x9F\x98\x87':	'smiling face with halo',
'\xF0\x9F\x98\x88':	'smiling face with horns',
'\xF0\x9F\x98\x8E':	'smiling face with sunglasses',
'\xF0\x9F\x98\x90':	'neutral face',
'\xF0\x9F\x98\x91':	'expressionless face',
'\xF0\x9F\x98\x95':	'confused face',
'\xF0\x9F\x98\x97':	'kissing face',
'\xF0\x9F\x98\x99':	'kissing face with smiling eyes',
'\xF0\x9F\x98\x9B':	'face with stuck-out tongue',
'\xF0\x9F\x98\x9F':	'worried face',
'\xF0\x9F\x98\xA6':	'frowning face with open mouth',
'\xF0\x9F\x98\xA7':	'anguished face',
'\xF0\x9F\x98\xAC':	'grimacing face',
'\xF0\x9F\x98\xAE':	'face with open mouth',
'\xF0\x9F\x98\xAF':	'hushed face',
'\xF0\x9F\x98\xB4':	'sleeping face',
'\xF0\x9F\x98\xB6':	'face without mouth',
'\xF0\x9F\x9A\x81':	'helicopter',
'\xF0\x9F\x9A\x82':	'steam locomotive',
'\xF0\x9F\x9A\x86':	'train',
'\xF0\x9F\x9A\x88':	'light rail',
'\xF0\x9F\x9A\x8A':	'tram',
'\xF0\x9F\x9A\x8D':	'oncoming bus',
'\xF0\x9F\x9A\x8E':	'trolleybus',
'\xF0\x9F\x9A\x90':	'minibus',
'\xF0\x9F\x9A\x94':	'oncoming police car',
'\xF0\x9F\x9A\x96':	'oncoming taxi',
'\xF0\x9F\x9A\x98':	'oncoming automobile',
'\xF0\x9F\x9A\x9B':	'articulated lorry',
'\xF0\x9F\x9A\x9C':	'tractor',
'\xF0\x9F\x9A\x9D':	'monorail',
'\xF0\x9F\x9A\x9E':	'mountain railway',
'\xF0\x9F\x9A\x9F':	'suspension railway',
'\xF0\x9F\x9A\xA0':	'mountain cableway',
'\xF0\x9F\x9A\xA1':	'aerial tramway',
'\xF0\x9F\x9A\xA3':	'rowboat',
'\xF0\x9F\x9A\xA6':	'vertical traffic light',
'\xF0\x9F\x9A\xAE':	'put litter in its place symbol',
'\xF0\x9F\x9A\xAF':	'do not litter symbol',
'\xF0\x9F\x9A\xB0':	'potable water symbol',
'\xF0\x9F\x9A\xB1':	'non-potable water symbol',
'\xF0\x9F\x9A\xB3':	'no bicycles',
'\xF0\x9F\x9A\xB4':	'bicyclist',
'\xF0\x9F\x9A\xB5':	'mountain bicyclist',
'\xF0\x9F\x9A\xB7':	'no pedestrians',
'\xF0\x9F\x9A\xB8':	'children crossing',
'\xF0\x9F\x9A\xBF':	'shower',
'\xF0\x9F\x9B\x81':	'bathtub',
'\xF0\x9F\x9B\x82':	'passport control',
'\xF0\x9F\x9B\x83':	'customs',
'\xF0\x9F\x9B\x84':	'baggage claim',
'\xF0\x9F\x9B\x85':	'left luggage',
'\xF0\x9F\x8C\x8D':	'earth globe europe-africa',
'\xF0\x9F\x8C\x8E':	'earth globe americas',
'\xF0\x9F\x8C\x90':	'globe with meridians',
'\xF0\x9F\x8C\x92':	'waxing crescent moon symbol',
'\xF0\x9F\x8C\x96':	'waning gibbous moon symbol',
'\xF0\x9F\x8C\x97':	'last quarter moon symbol',
'\xF0\x9F\x8C\x98':	'waning crescent moon symbol',
'\xF0\x9F\x8C\x9A':	'new moon with face',
'\xF0\x9F\x8C\x9C':	'last quarter moon with face',
'\xF0\x9F\x8C\x9D':	'full moon with face',
'\xF0\x9F\x8C\x9E':	'sun with face',
'\xF0\x9F\x8C\xB2':	'evergreen tree',
'\xF0\x9F\x8C\xB3':	'deciduous tree',
'\xF0\x9F\x8D\x8B':	'lemon',
'\xF0\x9F\x8D\x90':	'pear',
'\xF0\x9F\x8D\xBC':	'baby bottle',
'\xF0\x9F\x8F\x87':	'horse racing',
'\xF0\x9F\x8F\x89': 'rugby football',
'\xF0\x9F\x8F\xA4':	'european post office',
'\xF0\x9F\x90\x80':	'rat',
'\xF0\x9F\x90\x81':	'mouse',
'\xF0\x9F\x90\x82':	'ox',
'\xF0\x9F\x90\x83':	'water buffalo',
'\xF0\x9F\x90\x84':	'cow',
'\xF0\x9F\x90\x85':	'tiger',
'\xF0\x9F\x90\x86':	'leopard',
'\xF0\x9F\x90\x87':	'rabbit',
'\xF0\x9F\x90\x88':	'cat',
'\xF0\x9F\x90\x89':	'dragon',
'\xF0\x9F\x90\x8A':	'crocodile',
'\xF0\x9F\x90\x8B':	'whale',
'\xF0\x9F\x90\x8F':	'ram',
'\xF0\x9F\x90\x90':	'goat',
'\xF0\x9F\x90\x93':	'rooster',
'\xF0\x9F\x90\x95':	'dog',
'\xF0\x9F\x90\x96':	'pig',
'\xF0\x9F\x90\xAA':	'dromedary camel',
'\xF0\x9F\x91\xA5':	'busts in silhouette',
'\xF0\x9F\x91\xAC':	'two men holding hands',
'\xF0\x9F\x91\xAD':	'two women holding hands',
'\xF0\x9F\x92\xAD':	'thought balloon',
'\xF0\x9F\x92\xB6':	'banknote with euro sign',
'\xF0\x9F\x92\xB7':	'banknote with pound sign',
'\xF0\x9F\x93\xAC':	'open mailbox with raised flag',
'\xF0\x9F\x93\xAD':	'open mailbox with lowered flag',
'\xF0\x9F\x93\xAF':	'postal horn',
'\xF0\x9F\x93\xB5':	'no mobile phones',
'\xF0\x9F\x94\x80':	'twisted rightwards arrows',
'\xF0\x9F\x94\x81':	'clockwise rightwards and leftwards open circle arrows',
'\xF0\x9F\x94\x82':	'clockwise rightwards and leftwards open circle arrows with circled one overlay',
'\xF0\x9F\x94\x84':	'anticlockwise downwards and upwards open circle arrows',
'\xF0\x9F\x94\x85': 'low brightness symbol',
'\xF0\x9F\x94\x86':	'high brightness symbol',
'\xF0\x9F\x94\x87':	'speaker with cancellation stroke',
'\xF0\x9F\x94\x89':	'speaker with one sound wave',
'\xF0\x9F\x94\x95':	'bell with cancellation stroke',
'\xF0\x9F\x94\xAC':	'microscope',
'\xF0\x9F\x94\xAD':	'telescope',
'\xF0\x9F\x95\x9C':	'clock face one-thirty',
'\xF0\x9F\x95\x9D':	'clock face two-thirty',
'\xF0\x9F\x95\x9E':	'clock face three-thirty',
'\xF0\x9F\x95\x9F':	'clock face four-thirty',
'\xF0\x9F\x95\xA0':	'clock face five-thirty',
'\xF0\x9F\x95\xA1':	'clock face six-thirty',
'\xF0\x9F\x95\xA2':	'clock face seven-thirty',
'\xF0\x9F\x95\xA3':	'clock face eight-thirty',
'\xF0\x9F\x95\xA4':	'clock face nine-thirty',
'\xF0\x9F\x95\xA5':	'clock face ten-thirty',
'\xF0\x9F\x95\xA6':	'clock face eleven-thirty',
'\xF0\x9F\x95\xA7':	'clock face twelve-thirty'
}

In [16]:
# 5. Замена эмотиконов (пример: ":)" = "happy") на пробелы с использованием emoticon_dict.
df['tweet'] = df['tweet'].apply(lambda x: replace_words(x, emoticon_dict))

In [17]:
def replace_punctuation(text, pattern_for_punctuation):
  return re.sub(pattern_for_punctuation, ' ', text)

In [18]:
# 6. Замена пунктуации на пробелы с использованием re.sub() и паттерна r'[^\w\s]'.
pattern_for_punctuation = r'[^\w\s]'
df['tweet'] = df['tweet'].apply(lambda x: delete_pattern(x, pattern_for_punctuation))

In [19]:
def replace_special_symb(text, pattern_for_special_symb):
  return re.sub(pattern_for_special_symb, ' ', text)

In [20]:
# 7. Замена спец. символов на пробелы с использованием re.sub() и паттерна r'[^a-zA-Z0-9]'.
pattern_for_special_symb = r'[^a-zA-Z0-9]'
df['tweet'] = df['tweet'].apply(lambda x: delete_pattern(x, pattern_for_special_symb))

In [21]:
def delete_short_words(text, min_len):
  return ' '.join([word for word in text.split() if len(word) > min_len])

In [22]:
# 8. Замена чисел на пробелы с использованием re.sub() и паттерна r'[^a-zA-Z]'.
pattern_for_numbers = r'[^a-zA-Z]'
df['tweet'] = df['tweet'].apply(lambda x: delete_pattern(x, pattern_for_special_symb))

In [23]:
# 9. Удаление из текста слова длиной в 1 символ с использованием конструкции ' '.join([w for w in x.split() if len(w)>1]).
min_len = 1
df['tweet'] = df['tweet'].apply(lambda x: delete_short_words(x, min_len))

In [24]:
df.head()

,id,label,tweet
0,1,0.0,when father is dysfunctional and is so selfish...
1,2,0.0,thanks for lyft credit cannot use cause they d...
2,3,0.0,bihday your majesty
3,4,0.0,model love you take with you all the time in ur
4,5,0.0,factsguide society now motivation


In [25]:
# 10. Деление твитов на токены с помощью nltk.tokenize.word_tokenize с созданием нового столбца 'tweet_token'.
df['tweet_token'] = df['tweet'].apply(lambda x: tknz.word_tokenize(x))

In [26]:
df.head()

,id,label,tweet,tweet_token
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]"
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ..."
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]"


In [27]:
def delete_stop_words(text, stop_words):
  return ' '.join([word for word in text if word not in stop_words])

In [28]:
# 11. Удаление стоп-слов из токенов с использованием nltk.corpus.stopwords. Создание столбца 'tweet_token_filtered' без стоп-слов.
stop_words = set(stopwords.words("russian"))
df['tweet_token_filtered'] = df['tweet_token'].apply(lambda x: delete_stop_words(x, stop_words))

In [29]:
def compare_stemmer(text, stemmer):
  return ' '.join([stemmer.stem(word) for word in text.split()])

In [30]:
# 12. Применение стемминга к токенам с помощью nltk.stem.PorterStemmer. Создание столбца 'tweet_stemmed' после применения стемминга.
stemmer = PorterStemmer()
df['tweet_stemmed'] = df['tweet_token_filtered'].apply(lambda x: compare_stemmer(x, stemmer))

In [31]:
def compare_lemmatizer(text, lemmatizer):
  return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [32]:
# 13. Применение лемматизации к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создание столбца 'tweet_lemmatized' после применения лемматизации.
lemmatizer = WordNetLemmatizer()
df['tweet_lemmatized'] = df['tweet_token_filtered'].apply(lambda x: compare_lemmatizer(x, lemmatizer))

In [33]:
df.head(10)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...",when father is dysfunctional and is so selfish...,when father is dysfunct and is so selfish he d...,when father is dysfunctional and is so selfish...
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...",thanks for lyft credit can not use cause they ...,thank for lyft credit can not use caus they do...,thanks for lyft credit can not use cause they ...
2,3,0.0,bihday your majesty,"[bihday, your, majesty]",bihday your majesty,bihday your majesti,bihday your majesty
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...",model love you take with you all the time in ur,model love you take with you all the time in ur,model love you take with you all the time in ur
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]",factsguide society now motivation,factsguid societi now motiv,factsguide society now motivation
5,6,0.0,huge fan fare and big talking before they leav...,"[huge, fan, fare, and, big, talking, before, t...",huge fan fare and big talking before they leav...,huge fan fare and big talk befor they leav cha...,huge fan fare and big talking before they leav...
6,7,0.0,camping tomorrow danny,"[camping, tomorrow, danny]",camping tomorrow danny,camp tomorrow danni,camping tomorrow danny
7,8,0.0,the next school year is the year for exams can...,"[the, next, school, year, is, the, year, for, ...",the next school year is the year for exams can...,the next school year is the year for exam can ...,the next school year is the year for exam can ...
8,9,0.0,we won love the land allin cavs champions clev...,"[we, won, love, the, land, allin, cavs, champi...",we won love the land allin cavs champions clev...,we won love the land allin cav champion clevel...,we won love the land allin cavs champion cleve...
9,10,0.0,welcome here am it has it is so gr8,"[welcome, here, am, it, has, it, is, so, gr8]",welcome here am it has it is so gr8,welcom here am it ha it is so gr8,welcome here am it ha it is so gr8


In [34]:
# 14. Сохранение результата предобработки в pickle-файл.
with open('NLP_HW_1_results.pickle', 'wb') as f:
    pickle.dump(df, f)

In [35]:
with open('NLP_HW_1_results.pickle', 'rb') as f:
    df_check = pickle.load(f, encoding='utf-8')

In [36]:
df_check.head()

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...",when father is dysfunctional and is so selfish...,when father is dysfunct and is so selfish he d...,when father is dysfunctional and is so selfish...
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...",thanks for lyft credit can not use cause they ...,thank for lyft credit can not use caus they do...,thanks for lyft credit can not use cause they ...
2,3,0.0,bihday your majesty,"[bihday, your, majesty]",bihday your majesty,bihday your majesti,bihday your majesty
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...",model love you take with you all the time in ur,model love you take with you all the time in ur,model love you take with you all the time in ur
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]",factsguide society now motivation,factsguid societi now motiv,factsguide society now motivation
